---

# Atenção!

Lembre-se de clonar este notebook antes de tentar editar as células de código.

Para isso, basta seguir os passos:

File -> Save a copy in Drive



---

## Exercício 1

In [ ]:
from itertools import permutations

import numpy as np


def solve_tsp_brute_force(distance_matrix):
    """Resolve o TSP por meio de um algoritmo de força bruta"""
    n = distance_matrix.shape[0]  # número de nós do problema
    best_route = None  # inicializa melhor rota como vazio
    best_route_distance = np.inf  # distância inicial é um número muito alto

    for permutation in permutations(range(1, n)):
        route = [0] + list(permutation) + [0]
        route_distance = compute_route_distance(distance_matrix, route)

        # Se esta permutação possuir uma distância total menor que a atual,
        # substitua por ela
        if route_distance < best_route_distance:
            best_route = route
            best_route_distance = route_distance

    return best_route, best_route_distance

def compute_route_distance(distance_matrix, route):
    """Calcula a distância total de uma rota para uma dada matriz de distâncias"""
    route_distance = 0
    for i in range(len(route) - 1):
        route_distance += distance_matrix[route[i], route[i + 1]]

    return route_distance

O passo mais importante consiste em determinar a matriz de distâncias. Em seguida, basta aplicar nosso solver de força bruta.

In [ ]:
distance_matrix = np.array(
    [
        [0, 7, 7, 8, 5, 6],
        [7, 0, 5, 10, 22, 5],
        [7, 5, 0, 6, 16, 15],
        [8, 10, 6, 0, 6, 20],
        [5, 22, 16, 6, 0, 11],
        [6, 5, 15, 20, 11, 0],
    ]
)

solve_tsp_brute_force(distance_matrix)

# Exercício 2


Usando a dica, podemos primeiro recalcular a matriz de distâncias:

In [ ]:
distance_matrix = np.array(
    [
        [0, 7, 7, 8, 5, 6],
        [7, 0, 5, 10, 22, 5],
        [7, 5, 0, 6, 16, 15],
        [8, 10, 6, 0, 6, 20],
        [5, 22, 16, 6, 0, 11],
        [6, 5, 15, 20, 11, 0],
    ]
)

n = distance_matrix.shape[0]
for i in range(n - 1):
    distance_matrix[i, i + 1] = round(0.9 * distance_matrix[i + 1, i])

# Para conferir:
for i in range(n - 1):
    print(f"Ida: {distance_matrix[i, i + 1]}, Volta: {distance_matrix[i + 1, i]}")

In [ ]:
# Com isso, aqui está a solução
solve_tsp_brute_force(distance_matrix)

Observe como na solução do exercício 1 a rota consistia em nós decrescentes ([0, 4, 3, 2, 1, 5, 0]) -- ou seja, o caminho "voltando" era mais eficiente, enquanto, neste caso, houve uma troca já que vale mais a pena "ir" ([0, 5, 1, 2, 3, 4, 0]). Estas diferenças sutis são importantes, principalmente quando tratamos de rotas em cidades.


# Exercício 3

Aqui está um código que calcula este tempo para cada valor de `n`.

In [ ]:
from timeit import default_timer

import numpy as np


TIME_THRESHOLD_S = 60

results = [
    {
        "problem_size": 0,
        "processing_time": 0,
        "route_distance": 0,
    }
]
n = 3

while results[-1]["processing_time"] <= TIME_THRESHOLD_S:
    print(f"Problema com tamanho {n}")

    # Fixe o gerador de números aleatórios para obter a mesma matriz de
    # distâncias em cada execução
    np.random.seed(1)
    distance_matrix = np.random.randint(0, 21, size=(n, n))
    tic = default_timer()
    _, route_distance = solve_tsp_brute_force(distance_matrix)
    processing_time = default_timer() - tic

    results.append({
        "problem_size": n,
        "processing_time": processing_time,
        "route_distance": route_distance,
    })
    n += 1

results

Neste caso, problemas com 12 ou mais nós -- ou seja, um veículo saindo do ponto de partida e fazendo 11 entregas -- já deixam de ser factíveis para o nosso limite de um minuto. Uma vez que é comum um veículo entregar 50 ou mais pacotes, este algoritmo não poderia ser empregado em um sistema de larga escala.

Observe também como o tempo dá saltos enormes quando o tamanho do problema aumenta em apenas uma unidade.



# Exercício 4


In [ ]:
!pip install ortools==9.0.9048

In [ ]:
import numpy as np
from ortools.constraint_solver import pywrapcp


def solve_tsp_ortools(distance_matrix):
    n = distance_matrix.shape[0]  # número de nós do problema
    num_vehicles = 1  # número de veículos (no nosso caso, apenas um)
    depot_node = 0  # número do nó que representa o ponto de origem
    manager = pywrapcp.RoutingIndexManager(n, num_vehicles, depot_node)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(i, j):
        # `i` e `j` são índices internos do OR-Tools. Precisamos primeiro
        # convertê-los em nós do nosso problema
        ni = manager.IndexToNode(i)
        nj = manager.IndexToNode(j)
        return distance_matrix[ni, nj]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Resolve o problema com métodos default
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # Constroi a rota final
    route = []
    index = routing.Start(0)
    node = manager.IndexToNode(index)
    route.append(node)

    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        node = manager.IndexToNode(index)
        route.append(node)

    return route, solution.ObjectiveValue()

# Exercício 1
distance_matrix1 = np.array(
    [
        [0, 7, 7, 8, 5, 6],
        [7, 0, 5, 10, 22, 5],
        [7, 5, 0, 6, 16, 15],
        [8, 10, 6, 0, 6, 20],
        [5, 22, 16, 6, 0, 11],
        [6, 5, 15, 20, 11, 0],
    ]
)
route1, route_distance1 = solve_tsp_ortools(distance_matrix1)

print(f"Exercício 1 resolvido pelo OR-Tools: rota {route1}, distância: {route_distance1}")

# Exercício 2
distance_matrix2 = distance_matrix1.copy()
n = distance_matrix2.shape[0]
for i in range(n - 1):
    distance_matrix2[i, i + 1] = round(0.9 * distance_matrix2[i + 1, i])

route2, route_distance2 = solve_tsp_ortools(distance_matrix2)

print(f"Exercício 2 resolvido pelo OR-Tools: rota {route2}, distância: {route_distance2}")

Como se vê, nestes problemas pequenos o OR-Tools encontrou a solução ótima quando comparado ao nosso solver de força bruta.

# Exercício 5

Aqui está um código similar ao anterior, mas usando o OR-Tools até `n = 12`, que foi o limite encontrado no Exercício 3.

In [ ]:
from timeit import default_timer

import numpy as np


results = [
    {
        "problem_size": 0,
        "processing_time": 0,
        "route_distance": 0,
    }
]

for n in range(3, 13):
    print(f"Problema com tamanho {n}")

    # Fixe o gerador de números aleatórios para obter a mesma matriz de
    # distâncias em cada execução
    np.random.seed(1)
    distance_matrix = np.random.randint(0, 21, size=(n, n))
    tic = default_timer()
    _, route_distance = solve_tsp_ortools(distance_matrix)
    processing_time = default_timer() - tic

    results.append({
        "problem_size": n,
        "processing_time": processing_time,
        "route_distance": route_distance,
    })

results

Observe como os tempos de execução são mais ou menos constantes apesar do aumento de `n`, diferentemente do solver de força bruta que cresce muito mais rápido.

Em termos de qualidade, como resolvemos o mesmo problema, podemos ver que a partir de `n = 6` a heurística do OR-Tools deixa de retornar a melhor solução. Lembre que isso é aceitável, mas é possível melhorar seu desempenho trocando o algoritmo usado. Consulte [a documentação](https://developers.google.com/optimization/routing/tsp) e experimente resolver novamente este exercício.

Curiosamente, para problemas menores (até `n = 7` neste nosso teste), o algoritmo de força bruta é mais rápido ou comparável. Isto provavelmente se dá pelo fato de o OR-Tools precisar chamar um outro algoritmo em C internamente, enquanto o primeiro não tem esta interface.